# MobiML GeoTrackNet Demo

Based on: https://github.com/CIA-Oceanix/GeoTrackNet (MIT Licensed, (c) 2018 Duong Nguyen)

As presented in Nguyen, D., Vadaine, R., Hajduch, G., Garello, R. (2022). GeoTrackNet - A Maritime Anomaly Detector Using Probabilistic Neural Network Representation of AIS Tracks and A Contrario Detection. In IEEE Transactions on Intelligent Transportation Systems, 23(6).


Using data from AISDK: http://web.ais.dk/aisdata/aisdk-2018-02.zip

*It is possible to further explore maritime traffic patterns with the TrAISformer (https://github.com/CIA-Oceanix/TrAISformer), which is used for vessel trajectory prediction. The TrAISformer can be trained with AIS data and the preprocessing steps are similar to those of GeoTrackNet. However, the TrAISformer is out of the scope of MobiML and is an optional extension for the user to explore.*

## Environments

### Preprocessing

It is recommended to perform the preprocessing steps with the MobiML environment.

### Model Training

Set up a dedicated GeoTrackNet environment (PY3GPU) to train the model as instructed by Nguyen et al. (2022).

## Preprocessing

In [ ]:
import numpy as np
import os
import sys
import pickle
from datetime import datetime
from tqdm import tqdm
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
from datetime import datetime, timedelta

sys.path.append("..")
from mobiml.datasets import AISDK
from mobiml.preprocessing import (
    TrajectorySplitter,
    TrajectoryFilter,
    TrajectorySubsampler,
)
from mobiml.transforms import TemporalSplitter

In [ ]:
%pwd

In [3]:
# AISDK dataset
LAT, LON, SOG, COG, NAME, SHIPTYPE, NAV_STT, TIMESTAMP, TRAJ_ID = list(range(9))

EPOCH = datetime(1970, 1, 1)

SOG_MIN = 2.0
SOG_MAX = 30.0  # SOG is truncated to 30.0 knots max

# Pkl filenames
pkl_filename_train = "aisdk_20180208_train.pkl"
pkl_filename_valid = "aisdk_20180208_valid.pkl"
pkl_filename_test = "aisdk_20180208_test.pkl"

# Path to csv files
data_path = "data/aisdk_20180208_sample/"
csv_filename = "aisdk_20180208_sample.csv"

# Output path
out_path = "data/aisdk_20180208_sample/"

### Loading data

In [ ]:
path = os.path.join(data_path, csv_filename)
print(f"{datetime.now()} Loading data from {path}")
aisdk = AISDK(path)  # you can specify a bounding box here to filter the area
LON_MIN, LAT_MIN, LON_MAX, LAT_MAX = aisdk.get_bounds()
print(
    f"Bounding box:\nmin_lon: {LON_MIN}\nmin_lat: {LAT_MIN}\nmax_lon: {LON_MAX}\nmax_lat: {LAT_MAX}"
)

### Cleaning

#### Remove missing values

In [ ]:
aisdk.df = aisdk.df.dropna()
aisdk.df

In [ ]:
print("After removing missing values we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])
print("Total number of vessels:", len(aisdk.df.traj_id.unique()))
print("Lat min: ", aisdk.df.y.min(), "Lat max: ", aisdk.df.y.max())
print("Lon min: ", aisdk.df.x.min(), "Lon max: ", aisdk.df.x.max())
print("Time min: ", aisdk.df.timestamp.min(), "Time max: ", aisdk.df.timestamp.max())

#### Remove 'Moored' and 'At anchor' AIS messages

In [ ]:
aisdk.df.drop(aisdk.df[(aisdk.df["nav_status"] == "Moored") | (aisdk.df["nav_status"] == "At anchor")].index, inplace=True)
print("After removing 'Moored' or 'At anchor' AIS messages we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Keep only 'Cargo', 'Tanker', 'Passenger' vessel types

In [ ]:
aisdk.df = aisdk.df[
    (aisdk.df["ship_type"] == "Cargo")
    | (aisdk.df["ship_type"] == "Tanker")
    | (aisdk.df["ship_type"] == "Passenger")
]
print("After keeping only 'Cargo', 'Tanker' or 'Passenger' AIS messages we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Split trajectories with observation gaps > 2 hrs

In [ ]:
aisdk = TrajectorySplitter(aisdk).split(observation_gap=timedelta(hours=2))
print("After splitting trajectories with observation gaps we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Drop trajectories with fewer than $Points_{min}$ locations

In [ ]:
aisdk = TrajectoryFilter(aisdk).filter_min_pts(min_pts=20)
print("After removing trajectories with too few points we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Drop speed outliers

In [ ]:
aisdk = TrajectoryFilter(aisdk).filter_speed(min_speed=SOG_MIN, max_speed=SOG_MAX)
print("After removing speed outliers by setting a minimum and maximum speed we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

In [12]:
tc = aisdk.to_trajs() #  mpd.TrajectoryCollection(aisdk.df, "traj_id", t="timestamp", x="x", y="y")
traj_gdf = tc.to_traj_gdf()

We may also want to remove trajectories based on their overall average speed rather than the SOG values:

In [13]:
for index, row in traj_gdf.iterrows():
    traj_gdf.loc[index, "speed_ok"] = (
        tc.trajectories[index].get_length()
        / tc.trajectories[index].get_duration().total_seconds()
        > 1.02889  # 2 knots
    )

In [14]:
traj_gdf = traj_gdf[traj_gdf["speed_ok"] == True]

In [ ]:
aisdk.df = pd.merge(aisdk.df, traj_gdf["traj_id"], how="inner")
print("After removing speed outliers based on length and duration we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

### Training data preparation
#### Subsample AIS tracks 

In [ ]:
aisdk = TrajectorySubsampler(aisdk).subsample(min_dt_sec=60)
print("After subsampling AIS tracks we have...")
print("Total number of AIS messages: ", aisdk.df.shape[0])

#### Temporal train/valid/test split

In [ ]:
aisdk = TemporalSplitter(aisdk).split_hr()
aisdk.df

In [ ]:
aisdk_train = aisdk.df[(aisdk.df["split"] == 1.0)]
aisdk_valid = aisdk.df[(aisdk.df["split"] == 2.0)]
aisdk_test = aisdk.df[(aisdk.df["split"] == 3.0)]

print("Total number of AIS messages: ", len(aisdk.df))
print("Number of msgs in the training set: ", len(aisdk_train))
print("Number of msgs in the validation set: ", len(aisdk_valid))
print("Number of msgs in the test set: ", len(aisdk_test))

In [ ]:
aisdk_train

In [ ]:
target_column_order=["y", "x", "speed", "direction", "Name", "ship_type", "nav_status", "timestamp", "traj_id"]
aisdk_train = aisdk_train[target_column_order].reset_index(drop=True)
aisdk_valid = aisdk_valid[target_column_order].reset_index(drop=True)
aisdk_test = aisdk_test[target_column_order].reset_index(drop=True)
aisdk_test

#### Format timestamp

In [ ]:
aisdk_train["timestamp"] = (aisdk_train["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_valid["timestamp"] = (aisdk_valid["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_test["timestamp"]  = (aisdk_test["timestamp"].astype(int) / 1_000_000_000).astype(int)
aisdk_test

#### Format to ndarrays

In [22]:
aisdk_train = np.array(aisdk_train)
aisdk_valid = np.array(aisdk_valid)
aisdk_test = np.array(aisdk_test)

#### Merging into dict
Creating AIS tracks from the list of AIS messages. Each AIS track is formatted by a dictionary.

In [ ]:
print("Convert to dicts of vessel's tracks...")

def convert_tracks_to_dicts(tracks):
    d = dict()
    for v_msg in tqdm(tracks):
        mmsi = int(v_msg[TRAJ_ID])
        if not (mmsi in list(d.keys())):
            d[mmsi] = np.empty((0, 9))
        d[mmsi] = np.concatenate(
            (d[mmsi], np.expand_dims(v_msg[:9], 0)), axis=0
        )
    for key in tqdm(list(d.keys())):
        d[key] = np.array(
            sorted(d[key], key=lambda m_entry: m_entry[TIMESTAMP])
        )
    return d

Vs_train = convert_tracks_to_dicts(aisdk_train)
Vs_valid = convert_tracks_to_dicts(aisdk_valid)
Vs_test = convert_tracks_to_dicts(aisdk_test)

#### Normalisation

In [ ]:
print("Normalising data ...")

def normalize(d):
    for k in tqdm(list(d.keys())):
        v = d[k]
        v[:, LAT] = (v[:, LAT] - LAT_MIN) / (LAT_MAX - LAT_MIN)
        v[:, LON] = (v[:, LON] - LON_MIN) / (LON_MAX - LON_MIN)
        v[:, SOG][v[:, SOG] > SOG_MAX] = SOG_MAX
        v[:, SOG] = v[:, SOG] / SOG_MAX
        v[:, COG] = v[:, COG] / 360.0
    return d 

Vs_train = normalize(Vs_train)
Vs_valid = normalize(Vs_valid)
Vs_test = normalize(Vs_test)

In [ ]:
for filename, filedict in zip(
    [pkl_filename_train, pkl_filename_valid, pkl_filename_test],
    [Vs_train, Vs_valid, Vs_test],
):
    print("Writing to", os.path.join(out_path, filename))
    with open(os.path.join(out_path, filename), "wb") as f:
        pickle.dump(filedict, f)

## Model Training

From this point forward, it is recommended to execute the code with the [PY3GPU environment](https://github.com/CIA-Oceanix/GeoTrackNet/blob/master/requirements.yml), as set up by Nguyen et al. (2022).

### Setup

In [1]:
import os

# AISDK dataset
LAT, LON, SOG, COG, NAME, SHIPTYPE, NAV_STT, TIMESTAMP, TRAJ_ID = list(range(9))

# Pkl filenames
pkl_filename_train = "aisdk_20180208_train.pkl"
pkl_filename_valid = "aisdk_20180208_valid.pkl"
pkl_filename_test = "aisdk_20180208_test.pkl"

data_path = "../examples/data/aisdk_20180208_sample/"
dataset_path = os.path.join(data_path, pkl_filename_train)

### Calculate AIS mean

In [ ]:
"""
Input pipelines script for Tensorflow graph.
This script is adapted from the original script of FIVO.
"""

import numpy as np
import pickle
import tensorflow as tf

LAT_BINS = 100
LON_BINS = 200
SOG_BINS = 30
COG_BINS = 72


def sparse_AIS_to_dense(msgs_, num_timesteps, mmsis):
    def create_dense_vect(msg, lat_bins=100, lon_bins=200, sog_bins=30, cog_bins=72):
        lat, lon, sog, cog = msg[0], msg[1], msg[2], msg[3]
        data_dim = lat_bins + lon_bins + sog_bins + cog_bins
        dense_vect = np.zeros(data_dim)
        dense_vect[int(lat * lat_bins)] = 1.0
        dense_vect[int(lon * lon_bins) + lat_bins] = 1.0
        dense_vect[int(sog * sog_bins) + lat_bins + lon_bins] = 1.0
        dense_vect[int(cog * cog_bins) + lat_bins + lon_bins + sog_bins] = 1.0
        return dense_vect

    dense_msgs = []
    for msg in msgs_:
        dense_msgs.append(
            create_dense_vect(
                msg,
                lat_bins=LAT_BINS,
                lon_bins=LON_BINS,
                sog_bins=SOG_BINS,
                cog_bins=COG_BINS,
            )
        )
    dense_msgs = np.array(dense_msgs)
    return dense_msgs, num_timesteps, mmsis


dirname = os.path.dirname(dataset_path)

try:
    with tf.gfile.Open(dataset_path, "rb") as f:
        Vs = pickle.load(f)
except:
    with tf.gfile.Open(dataset_path, "rb") as f:
        Vs = pickle.load(f, encoding="latin1")

data_dim = LAT_BINS + LON_BINS + SOG_BINS + COG_BINS

mean_all = np.zeros((data_dim,))
sum_all = np.zeros((data_dim,))
total_ais_msg = 0

current_mean = np.zeros((0, data_dim))
current_ais_msg = 0

count = 0
for mmsi in list(Vs.keys()):
    count += 1
    print(count)
    tmp = Vs[mmsi][:, [LAT, LON, SOG, COG]]
    tmp[tmp == 1] = 0.99999
    current_sparse_matrix, _, _ = sparse_AIS_to_dense(tmp, 0, 0)
    #    current_mean = np.mean(current_sparse_matrix,axis = 0)
    sum_all += np.sum(current_sparse_matrix, axis=0)
    total_ais_msg += len(current_sparse_matrix)

mean = sum_all / total_ais_msg

print("Writing to", os.path.join(dirname, "/mean.pkl"))
with open(dirname + "/mean.pkl", "wb") as f:
    pickle.dump(mean, f)

### Training

#### Step 1: Training the Embedding layer

In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import logging
import tensorflow as tf

import sys
sys.path.append("..")
import mobiml.models.geotracknet.runners as runners
from mobiml.models.geotracknet.flags_config import config

print(config.trainingset_path)
fh = logging.FileHandler(os.path.join(config.logdir, config.log_filename + ".log"))
tf.logging.set_verbosity(tf.logging.INFO)
# get TF logger
logger = logging.getLogger("tensorflow")
logger.addHandler(fh)
runners.run_train(config)

##### Command line version

`python geotracknet.py --mode=train --dataset_dir=../../../examples/data/aisdk_20180208_sample trainingset_name=aisdk_20180208_train.pkl --testset_name=aisdk_20180208_test.pkl --lat_min=57.0 --lat_max=58.0 --lon_min=11.0 --lon_max=13.0 --latent_size=100 --batch_size=32 --num_samples=16 --learning_rate=0.0003`

```
/home/anita/miniforge3/envs/PY3GPU/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
2024-10-21 21:19:31.109056: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2024-10-21 21:19:31.126958: E tensorflow/stream_executor/cuda/cuda_driver.cc:300] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-10-21 21:19:31.126981: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:150] kernel driver does not appear to be running on this host (anita-mint): /proc/driver/nvidia/version does not exist
INFO:tensorflow:Restoring parameters from ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:Step 1, elbo bound per timestep: -21.471972
INFO:tensorflow:global_step/sec: 3.80814
INFO:tensorflow:Step 101, elbo bound per timestep: -20.298294
INFO:tensorflow:global_step/sec: 3.83284
INFO:tensorflow:Step 201, elbo bound per timestep: -18.670677
INFO:tensorflow:global_step/sec: 3.73587
INFO:tensorflow:Step 301, elbo bound per timestep: -18.175743
INFO:tensorflow:global_step/sec: 3.71976
INFO:tensorflow:Step 401, elbo bound per timestep: -16.840940
INFO:tensorflow:Saving checkpoints for 450 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71769
INFO:tensorflow:Step 501, elbo bound per timestep: -14.940658
INFO:tensorflow:global_step/sec: 3.7761
INFO:tensorflow:Step 601, elbo bound per timestep: -13.312830
INFO:tensorflow:global_step/sec: 3.71831
INFO:tensorflow:Step 701, elbo bound per timestep: -12.890591
INFO:tensorflow:global_step/sec: 3.81016
INFO:tensorflow:Step 801, elbo bound per timestep: -12.362333
INFO:tensorflow:global_step/sec: 3.73768
INFO:tensorflow:Step 901, elbo bound per timestep: -10.832000
INFO:tensorflow:Saving checkpoints for 902 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68577
INFO:tensorflow:Step 1001, elbo bound per timestep: -11.429497
INFO:tensorflow:global_step/sec: 3.74095
INFO:tensorflow:Step 1101, elbo bound per timestep: -9.984019
INFO:tensorflow:global_step/sec: 3.76748
INFO:tensorflow:Step 1201, elbo bound per timestep: -9.857633
INFO:tensorflow:global_step/sec: 3.74169
INFO:tensorflow:Step 1301, elbo bound per timestep: -10.664632
INFO:tensorflow:Saving checkpoints for 1350 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.73368
INFO:tensorflow:Step 1401, elbo bound per timestep: -10.517320
INFO:tensorflow:global_step/sec: 3.73867
INFO:tensorflow:Step 1501, elbo bound per timestep: -9.585644
INFO:tensorflow:global_step/sec: 3.76114
INFO:tensorflow:Step 1601, elbo bound per timestep: -9.137579
INFO:tensorflow:global_step/sec: 3.73173
INFO:tensorflow:Step 1701, elbo bound per timestep: -9.256488
INFO:tensorflow:global_step/sec: 3.74943
INFO:tensorflow:Saving checkpoints for 1801 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:Step 1801, elbo bound per timestep: -9.426279
INFO:tensorflow:global_step/sec: 3.74547
INFO:tensorflow:Step 1901, elbo bound per timestep: -8.083561
INFO:tensorflow:global_step/sec: 3.77358
INFO:tensorflow:Step 2001, elbo bound per timestep: -7.876364
INFO:tensorflow:global_step/sec: 3.6601
INFO:tensorflow:Step 2101, elbo bound per timestep: -7.703875
INFO:tensorflow:global_step/sec: 3.73167
INFO:tensorflow:Step 2201, elbo bound per timestep: -7.446716
INFO:tensorflow:Saving checkpoints for 2249 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72816
INFO:tensorflow:Step 2301, elbo bound per timestep: -7.444590
INFO:tensorflow:global_step/sec: 3.70612
INFO:tensorflow:Step 2401, elbo bound per timestep: -7.719640
INFO:tensorflow:global_step/sec: 3.7968
INFO:tensorflow:Step 2501, elbo bound per timestep: -6.519677
INFO:tensorflow:global_step/sec: 3.7884
INFO:tensorflow:Step 2601, elbo bound per timestep: -6.851766
INFO:tensorflow:Saving checkpoints for 2699 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.70733
INFO:tensorflow:Step 2701, elbo bound per timestep: -6.559093
INFO:tensorflow:global_step/sec: 3.7835
INFO:tensorflow:Step 2801, elbo bound per timestep: -6.534922
INFO:tensorflow:global_step/sec: 3.75566
INFO:tensorflow:Step 2901, elbo bound per timestep: -6.508255
INFO:tensorflow:global_step/sec: 3.73277
INFO:tensorflow:Step 3001, elbo bound per timestep: -6.336797
INFO:tensorflow:global_step/sec: 3.71993
INFO:tensorflow:Step 3101, elbo bound per timestep: -5.583270
INFO:tensorflow:Saving checkpoints for 3148 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72718
INFO:tensorflow:Step 3201, elbo bound per timestep: -6.055767
INFO:tensorflow:global_step/sec: 3.74921
INFO:tensorflow:Step 3301, elbo bound per timestep: -5.967187
INFO:tensorflow:global_step/sec: 3.77162
INFO:tensorflow:Step 3401, elbo bound per timestep: -5.962825
INFO:tensorflow:global_step/sec: 3.77577
INFO:tensorflow:Step 3501, elbo bound per timestep: -6.110925
INFO:tensorflow:Saving checkpoints for 3599 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68564
INFO:tensorflow:Step 3601, elbo bound per timestep: -5.282250
INFO:tensorflow:global_step/sec: 3.79071
INFO:tensorflow:Step 3701, elbo bound per timestep: -4.511233
INFO:tensorflow:global_step/sec: 3.8372
INFO:tensorflow:Step 3801, elbo bound per timestep: -5.942684
INFO:tensorflow:global_step/sec: 3.69576
INFO:tensorflow:Step 3901, elbo bound per timestep: -5.614711
INFO:tensorflow:global_step/sec: 3.76596
INFO:tensorflow:Step 4001, elbo bound per timestep: -4.556732
INFO:tensorflow:Saving checkpoints for 4052 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72118
INFO:tensorflow:Step 4101, elbo bound per timestep: -5.070984
INFO:tensorflow:global_step/sec: 3.74082
INFO:tensorflow:Step 4201, elbo bound per timestep: -3.742627
INFO:tensorflow:global_step/sec: 3.7612
INFO:tensorflow:Step 4301, elbo bound per timestep: -4.000580
INFO:tensorflow:global_step/sec: 3.6901
INFO:tensorflow:Step 4401, elbo bound per timestep: -4.512469
INFO:tensorflow:Saving checkpoints for 4499 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68856
INFO:tensorflow:Step 4501, elbo bound per timestep: -4.029602
INFO:tensorflow:global_step/sec: 3.74513
INFO:tensorflow:Step 4601, elbo bound per timestep: -4.239000
INFO:tensorflow:global_step/sec: 3.72634
INFO:tensorflow:Step 4701, elbo bound per timestep: -4.819816
INFO:tensorflow:global_step/sec: 3.68273
INFO:tensorflow:Step 4801, elbo bound per timestep: -3.416077
INFO:tensorflow:global_step/sec: 3.76625
INFO:tensorflow:Step 4901, elbo bound per timestep: -4.365750
INFO:tensorflow:Saving checkpoints for 4947 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71903
INFO:tensorflow:Step 5001, elbo bound per timestep: -3.435213
INFO:tensorflow:global_step/sec: 3.69743
INFO:tensorflow:Step 5101, elbo bound per timestep: -3.434734
INFO:tensorflow:global_step/sec: 3.77223
INFO:tensorflow:Step 5201, elbo bound per timestep: -4.383901
INFO:tensorflow:global_step/sec: 3.81645
INFO:tensorflow:Step 5301, elbo bound per timestep: -2.744253
INFO:tensorflow:Saving checkpoints for 5397 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71099
INFO:tensorflow:Step 5401, elbo bound per timestep: -4.087787
INFO:tensorflow:global_step/sec: 3.70834
INFO:tensorflow:Step 5501, elbo bound per timestep: -3.509276
INFO:tensorflow:global_step/sec: 3.76402
INFO:tensorflow:Step 5601, elbo bound per timestep: -3.277967
INFO:tensorflow:global_step/sec: 3.70723
INFO:tensorflow:Step 5701, elbo bound per timestep: -3.517572
INFO:tensorflow:global_step/sec: 3.75295
INFO:tensorflow:Step 5801, elbo bound per timestep: -2.845486
INFO:tensorflow:Saving checkpoints for 5844 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71143
INFO:tensorflow:Step 5901, elbo bound per timestep: -2.664552
INFO:tensorflow:global_step/sec: 3.72077
INFO:tensorflow:Step 6001, elbo bound per timestep: -3.434897
INFO:tensorflow:global_step/sec: 3.77907
INFO:tensorflow:Step 6101, elbo bound per timestep: -2.988917
INFO:tensorflow:global_step/sec: 3.75856
INFO:tensorflow:Step 6201, elbo bound per timestep: -2.850919
INFO:tensorflow:Saving checkpoints for 6294 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68969
INFO:tensorflow:Step 6301, elbo bound per timestep: -3.338887
INFO:tensorflow:global_step/sec: 3.7234
INFO:tensorflow:Step 6401, elbo bound per timestep: -2.926769
INFO:tensorflow:global_step/sec: 3.75688
INFO:tensorflow:Step 6501, elbo bound per timestep: -2.519203
INFO:tensorflow:global_step/sec: 3.71921
INFO:tensorflow:Step 6601, elbo bound per timestep: -3.339667
INFO:tensorflow:global_step/sec: 3.74509
INFO:tensorflow:Step 6701, elbo bound per timestep: -2.264839
INFO:tensorflow:Saving checkpoints for 6743 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75806
INFO:tensorflow:Step 6801, elbo bound per timestep: -1.881311
INFO:tensorflow:global_step/sec: 3.75641
INFO:tensorflow:Step 6901, elbo bound per timestep: -2.346199
INFO:tensorflow:global_step/sec: 3.76239
INFO:tensorflow:Step 7001, elbo bound per timestep: -2.161547
INFO:tensorflow:global_step/sec: 3.76399
INFO:tensorflow:Step 7101, elbo bound per timestep: -2.790297
INFO:tensorflow:Saving checkpoints for 7197 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.86056
INFO:tensorflow:Step 7201, elbo bound per timestep: -2.802123
INFO:tensorflow:global_step/sec: 3.71902
INFO:tensorflow:Step 7301, elbo bound per timestep: -2.245830
INFO:tensorflow:global_step/sec: 3.68696
INFO:tensorflow:Step 7401, elbo bound per timestep: -2.067328
INFO:tensorflow:global_step/sec: 3.72243
INFO:tensorflow:Step 7501, elbo bound per timestep: -2.603912
INFO:tensorflow:global_step/sec: 3.7789
INFO:tensorflow:Step 7601, elbo bound per timestep: -2.003201
INFO:tensorflow:Saving checkpoints for 7645 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.7209
INFO:tensorflow:Step 7701, elbo bound per timestep: -2.083828
INFO:tensorflow:global_step/sec: 3.75109
INFO:tensorflow:Step 7801, elbo bound per timestep: -2.516495
INFO:tensorflow:global_step/sec: 3.84106
INFO:tensorflow:Step 7901, elbo bound per timestep: -2.018843
INFO:tensorflow:global_step/sec: 3.7362
INFO:tensorflow:Step 8001, elbo bound per timestep: -2.475885
INFO:tensorflow:Saving checkpoints for 8096 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69896
INFO:tensorflow:Step 8101, elbo bound per timestep: -2.242398
INFO:tensorflow:global_step/sec: 3.73859
INFO:tensorflow:Step 8201, elbo bound per timestep: -2.247642
INFO:tensorflow:global_step/sec: 3.75792
INFO:tensorflow:Step 8301, elbo bound per timestep: -1.802553
INFO:tensorflow:global_step/sec: 3.77683
INFO:tensorflow:Step 8401, elbo bound per timestep: -2.482219
INFO:tensorflow:global_step/sec: 3.74859
INFO:tensorflow:Step 8501, elbo bound per timestep: -1.877655
INFO:tensorflow:Saving checkpoints for 8547 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72188
INFO:tensorflow:Step 8601, elbo bound per timestep: -1.654466
INFO:tensorflow:global_step/sec: 3.82032
INFO:tensorflow:Step 8701, elbo bound per timestep: -1.813221
INFO:tensorflow:global_step/sec: 3.77956
INFO:tensorflow:Step 8801, elbo bound per timestep: -2.304314
INFO:tensorflow:global_step/sec: 3.75063
INFO:tensorflow:Step 8901, elbo bound per timestep: -2.026976
INFO:tensorflow:Saving checkpoints for 8998 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68476
INFO:tensorflow:Step 9001, elbo bound per timestep: -1.886283
INFO:tensorflow:global_step/sec: 3.77285
INFO:tensorflow:Step 9101, elbo bound per timestep: -2.045205
INFO:tensorflow:global_step/sec: 3.71099
INFO:tensorflow:Step 9201, elbo bound per timestep: -1.643228
INFO:tensorflow:global_step/sec: 3.74048
INFO:tensorflow:Step 9301, elbo bound per timestep: -1.636103
INFO:tensorflow:global_step/sec: 3.68953
INFO:tensorflow:Step 9401, elbo bound per timestep: -1.904341
INFO:tensorflow:Saving checkpoints for 9446 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.73057
INFO:tensorflow:Step 9501, elbo bound per timestep: -1.589882
INFO:tensorflow:global_step/sec: 3.75994
INFO:tensorflow:Step 9601, elbo bound per timestep: -1.949007
INFO:tensorflow:global_step/sec: 3.68059
INFO:tensorflow:Step 9701, elbo bound per timestep: -2.439717
INFO:tensorflow:global_step/sec: 3.79279
INFO:tensorflow:Step 9801, elbo bound per timestep: -1.169807
INFO:tensorflow:Saving checkpoints for 9897 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.79392
INFO:tensorflow:Step 9901, elbo bound per timestep: -1.775491
INFO:tensorflow:global_step/sec: 3.70566
INFO:tensorflow:Step 10001, elbo bound per timestep: -1.639194
INFO:tensorflow:global_step/sec: 3.73871
INFO:tensorflow:Step 10101, elbo bound per timestep: -1.622786
INFO:tensorflow:global_step/sec: 3.71306
INFO:tensorflow:Step 10201, elbo bound per timestep: -1.727285
INFO:tensorflow:global_step/sec: 3.73892
INFO:tensorflow:Step 10301, elbo bound per timestep: -1.790893
INFO:tensorflow:Saving checkpoints for 10345 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.76989
INFO:tensorflow:Step 10401, elbo bound per timestep: -1.380662
INFO:tensorflow:global_step/sec: 3.72416
INFO:tensorflow:Step 10501, elbo bound per timestep: -1.996268
INFO:tensorflow:global_step/sec: 3.74873
INFO:tensorflow:Step 10601, elbo bound per timestep: -1.384882
INFO:tensorflow:global_step/sec: 3.79214
INFO:tensorflow:Step 10701, elbo bound per timestep: -1.403177
INFO:tensorflow:Saving checkpoints for 10796 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.73949
INFO:tensorflow:Step 10801, elbo bound per timestep: -2.236970
INFO:tensorflow:global_step/sec: 3.75892
INFO:tensorflow:Step 10901, elbo bound per timestep: -1.292204
INFO:tensorflow:global_step/sec: 3.75928
INFO:tensorflow:Step 11001, elbo bound per timestep: -1.691911
INFO:tensorflow:global_step/sec: 3.77523
INFO:tensorflow:Step 11101, elbo bound per timestep: -1.853188
INFO:tensorflow:global_step/sec: 3.70781
INFO:tensorflow:Step 11201, elbo bound per timestep: -1.377297
INFO:tensorflow:Saving checkpoints for 11247 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.85093
INFO:tensorflow:Step 11301, elbo bound per timestep: -1.164689
INFO:tensorflow:global_step/sec: 3.72186
INFO:tensorflow:Step 11401, elbo bound per timestep: -1.523813
INFO:tensorflow:global_step/sec: 3.69341
INFO:tensorflow:Step 11501, elbo bound per timestep: -1.566616
INFO:tensorflow:global_step/sec: 3.75579
INFO:tensorflow:Step 11601, elbo bound per timestep: -1.379545
INFO:tensorflow:Saving checkpoints for 11697 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75259
INFO:tensorflow:Step 11701, elbo bound per timestep: -1.654644
INFO:tensorflow:global_step/sec: 3.7802
INFO:tensorflow:Step 11801, elbo bound per timestep: -1.497155
INFO:tensorflow:global_step/sec: 3.75434
INFO:tensorflow:Step 11901, elbo bound per timestep: -1.656492
INFO:tensorflow:global_step/sec: 3.76634
INFO:tensorflow:Step 12001, elbo bound per timestep: -1.290386
INFO:tensorflow:global_step/sec: 3.80532
INFO:tensorflow:Step 12101, elbo bound per timestep: -1.269332
INFO:tensorflow:Saving checkpoints for 12151 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.76285
INFO:tensorflow:Step 12201, elbo bound per timestep: -1.432564
INFO:tensorflow:global_step/sec: 3.72984
INFO:tensorflow:Step 12301, elbo bound per timestep: -1.180225
INFO:tensorflow:global_step/sec: 3.81057
INFO:tensorflow:Step 12401, elbo bound per timestep: -1.318207
INFO:tensorflow:global_step/sec: 3.72805
INFO:tensorflow:Step 12501, elbo bound per timestep: -1.598862
INFO:tensorflow:global_step/sec: 3.78618
INFO:tensorflow:Step 12601, elbo bound per timestep: -1.516943
INFO:tensorflow:Saving checkpoints for 12602 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75721
INFO:tensorflow:Step 12701, elbo bound per timestep: -0.885756
INFO:tensorflow:global_step/sec: 3.73075
INFO:tensorflow:Step 12801, elbo bound per timestep: -1.430757
INFO:tensorflow:global_step/sec: 3.74214
INFO:tensorflow:Step 12901, elbo bound per timestep: -1.140127
INFO:tensorflow:global_step/sec: 3.72515
INFO:tensorflow:Step 13001, elbo bound per timestep: -1.712610
INFO:tensorflow:Saving checkpoints for 13051 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.70882
INFO:tensorflow:Step 13101, elbo bound per timestep: -1.619936
INFO:tensorflow:global_step/sec: 3.75022
INFO:tensorflow:Step 13201, elbo bound per timestep: -1.564023
INFO:tensorflow:global_step/sec: 3.7427
INFO:tensorflow:Step 13301, elbo bound per timestep: -1.653574
INFO:tensorflow:global_step/sec: 3.71203
INFO:tensorflow:Step 13401, elbo bound per timestep: -0.956730
INFO:tensorflow:Saving checkpoints for 13499 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.73002
INFO:tensorflow:Step 13501, elbo bound per timestep: -1.389170
INFO:tensorflow:global_step/sec: 3.73458
INFO:tensorflow:Step 13601, elbo bound per timestep: -0.998512
INFO:tensorflow:global_step/sec: 3.76477
INFO:tensorflow:Step 13701, elbo bound per timestep: -1.489509
INFO:tensorflow:global_step/sec: 3.71135
INFO:tensorflow:Step 13801, elbo bound per timestep: -1.056787
INFO:tensorflow:global_step/sec: 3.57609
INFO:tensorflow:Step 13901, elbo bound per timestep: -1.314814
INFO:tensorflow:Saving checkpoints for 13941 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.66733
INFO:tensorflow:Step 14001, elbo bound per timestep: -1.023015
INFO:tensorflow:global_step/sec: 3.78448
INFO:tensorflow:Step 14101, elbo bound per timestep: -1.112222
INFO:tensorflow:global_step/sec: 3.66761
INFO:tensorflow:Step 14201, elbo bound per timestep: -1.273076
INFO:tensorflow:global_step/sec: 3.73883
INFO:tensorflow:Step 14301, elbo bound per timestep: -1.335328
INFO:tensorflow:Saving checkpoints for 14390 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.70192
INFO:tensorflow:Step 14401, elbo bound per timestep: -1.200685
INFO:tensorflow:global_step/sec: 3.7418
INFO:tensorflow:Step 14501, elbo bound per timestep: -1.107151
INFO:tensorflow:global_step/sec: 3.74526
INFO:tensorflow:Step 14601, elbo bound per timestep: -1.144458
INFO:tensorflow:global_step/sec: 3.71364
INFO:tensorflow:Step 14701, elbo bound per timestep: -1.096011
INFO:tensorflow:global_step/sec: 3.65574
INFO:tensorflow:Step 14801, elbo bound per timestep: -1.143260
INFO:tensorflow:Saving checkpoints for 14835 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72669
INFO:tensorflow:Step 14901, elbo bound per timestep: -1.155460
INFO:tensorflow:global_step/sec: 3.78208
INFO:tensorflow:Step 15001, elbo bound per timestep: -1.345529
INFO:tensorflow:global_step/sec: 3.74033
INFO:tensorflow:Step 15101, elbo bound per timestep: -1.116526
INFO:tensorflow:global_step/sec: 3.76882
INFO:tensorflow:Step 15201, elbo bound per timestep: -1.029002
INFO:tensorflow:Saving checkpoints for 15285 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.67196
INFO:tensorflow:Step 15301, elbo bound per timestep: -1.322693
INFO:tensorflow:global_step/sec: 3.76253
INFO:tensorflow:Step 15401, elbo bound per timestep: -1.104784
INFO:tensorflow:global_step/sec: 3.65508
INFO:tensorflow:Step 15501, elbo bound per timestep: -0.980326
INFO:tensorflow:global_step/sec: 3.68815
INFO:tensorflow:Step 15601, elbo bound per timestep: -1.209152
INFO:tensorflow:global_step/sec: 3.74377
INFO:tensorflow:Step 15701, elbo bound per timestep: -0.991307
INFO:tensorflow:Saving checkpoints for 15731 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.77737
INFO:tensorflow:Step 15801, elbo bound per timestep: -1.369320
INFO:tensorflow:global_step/sec: 3.72161
INFO:tensorflow:Step 15901, elbo bound per timestep: -0.942783
INFO:tensorflow:global_step/sec: 3.7251
INFO:tensorflow:Step 16001, elbo bound per timestep: -1.254408
INFO:tensorflow:global_step/sec: 3.7637
INFO:tensorflow:Step 16101, elbo bound per timestep: -1.102539
INFO:tensorflow:Saving checkpoints for 16180 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69512
INFO:tensorflow:Step 16201, elbo bound per timestep: -1.231005
INFO:tensorflow:global_step/sec: 3.74173
INFO:tensorflow:Step 16301, elbo bound per timestep: -0.902323
INFO:tensorflow:global_step/sec: 3.64351
INFO:tensorflow:Step 16401, elbo bound per timestep: -0.954200
INFO:tensorflow:global_step/sec: 3.71129
INFO:tensorflow:Step 16501, elbo bound per timestep: -0.876672
INFO:tensorflow:global_step/sec: 3.72205
INFO:tensorflow:Step 16601, elbo bound per timestep: -1.135966
INFO:tensorflow:Saving checkpoints for 16625 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.7409
INFO:tensorflow:Step 16701, elbo bound per timestep: -1.053402
INFO:tensorflow:global_step/sec: 3.75499
INFO:tensorflow:Step 16801, elbo bound per timestep: -0.970127
INFO:tensorflow:global_step/sec: 3.71703
INFO:tensorflow:Step 16901, elbo bound per timestep: -0.983622
INFO:tensorflow:global_step/sec: 3.77236
INFO:tensorflow:Step 17001, elbo bound per timestep: -1.132393
INFO:tensorflow:Saving checkpoints for 17076 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75511
INFO:tensorflow:Step 17101, elbo bound per timestep: -1.231874
INFO:tensorflow:global_step/sec: 3.6636
INFO:tensorflow:Step 17201, elbo bound per timestep: -1.441361
INFO:tensorflow:global_step/sec: 3.78019
INFO:tensorflow:Step 17301, elbo bound per timestep: -0.760114
INFO:tensorflow:global_step/sec: 3.77511
INFO:tensorflow:Step 17401, elbo bound per timestep: -1.287088
INFO:tensorflow:global_step/sec: 3.76618
INFO:tensorflow:Step 17501, elbo bound per timestep: -1.168090
INFO:tensorflow:Saving checkpoints for 17525 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71786
INFO:tensorflow:Step 17601, elbo bound per timestep: -0.969564
INFO:tensorflow:global_step/sec: 3.80349
INFO:tensorflow:Step 17701, elbo bound per timestep: -0.924661
INFO:tensorflow:global_step/sec: 3.7308
INFO:tensorflow:Step 17801, elbo bound per timestep: -0.825947
INFO:tensorflow:global_step/sec: 3.70895
INFO:tensorflow:Step 17901, elbo bound per timestep: -1.003554
INFO:tensorflow:Saving checkpoints for 17974 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.76906
INFO:tensorflow:Step 18001, elbo bound per timestep: -0.774617
INFO:tensorflow:global_step/sec: 3.74378
INFO:tensorflow:Step 18101, elbo bound per timestep: -1.194042
INFO:tensorflow:global_step/sec: 3.70318
INFO:tensorflow:Step 18201, elbo bound per timestep: -0.906552
INFO:tensorflow:global_step/sec: 3.74021
INFO:tensorflow:Step 18301, elbo bound per timestep: -0.881326
INFO:tensorflow:global_step/sec: 3.70247
INFO:tensorflow:Step 18401, elbo bound per timestep: -0.856460
INFO:tensorflow:Saving checkpoints for 18422 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.6724
INFO:tensorflow:Step 18501, elbo bound per timestep: -0.849353
INFO:tensorflow:global_step/sec: 3.76899
INFO:tensorflow:Step 18601, elbo bound per timestep: -1.089631
INFO:tensorflow:global_step/sec: 3.67558
INFO:tensorflow:Step 18701, elbo bound per timestep: -0.893693
INFO:tensorflow:global_step/sec: 3.78894
INFO:tensorflow:Step 18801, elbo bound per timestep: -0.809898
INFO:tensorflow:Saving checkpoints for 18869 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68516
INFO:tensorflow:Step 18901, elbo bound per timestep: -0.983065
INFO:tensorflow:global_step/sec: 3.70312
INFO:tensorflow:Step 19001, elbo bound per timestep: -1.022728
INFO:tensorflow:global_step/sec: 3.79567
INFO:tensorflow:Step 19101, elbo bound per timestep: -0.976628
INFO:tensorflow:global_step/sec: 3.70578
INFO:tensorflow:Step 19201, elbo bound per timestep: -0.778116
INFO:tensorflow:global_step/sec: 3.75925
INFO:tensorflow:Step 19301, elbo bound per timestep: -1.091054
INFO:tensorflow:Saving checkpoints for 19319 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69959
INFO:tensorflow:Step 19401, elbo bound per timestep: -0.708940
INFO:tensorflow:global_step/sec: 3.70521
INFO:tensorflow:Step 19501, elbo bound per timestep: -1.031905
INFO:tensorflow:global_step/sec: 3.76009
INFO:tensorflow:Step 19601, elbo bound per timestep: -0.944829
INFO:tensorflow:global_step/sec: 3.74073
INFO:tensorflow:Step 19701, elbo bound per timestep: -0.930751
INFO:tensorflow:Saving checkpoints for 19765 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.66344
INFO:tensorflow:Step 19801, elbo bound per timestep: -0.679564
INFO:tensorflow:global_step/sec: 3.70877
INFO:tensorflow:Step 19901, elbo bound per timestep: -0.756900
INFO:tensorflow:global_step/sec: 3.80147
INFO:tensorflow:Step 20001, elbo bound per timestep: -1.256270
INFO:tensorflow:global_step/sec: 3.77658
INFO:tensorflow:Step 20101, elbo bound per timestep: -1.162310
INFO:tensorflow:global_step/sec: 3.69452
INFO:tensorflow:Step 20201, elbo bound per timestep: -0.690918
INFO:tensorflow:Saving checkpoints for 20214 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69443
INFO:tensorflow:Step 20301, elbo bound per timestep: -1.077670
INFO:tensorflow:global_step/sec: 3.72301
INFO:tensorflow:Step 20401, elbo bound per timestep: -0.662932
INFO:tensorflow:global_step/sec: 3.74186
INFO:tensorflow:Step 20501, elbo bound per timestep: -0.734088
INFO:tensorflow:global_step/sec: 3.74151
INFO:tensorflow:Step 20601, elbo bound per timestep: -0.843539
INFO:tensorflow:Saving checkpoints for 20662 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.7427
INFO:tensorflow:Step 20701, elbo bound per timestep: -0.791845
INFO:tensorflow:global_step/sec: 3.68995
INFO:tensorflow:Step 20801, elbo bound per timestep: -0.815069
INFO:tensorflow:global_step/sec: 3.74091
INFO:tensorflow:Step 20901, elbo bound per timestep: -0.738795
INFO:tensorflow:global_step/sec: 3.71051
INFO:tensorflow:Step 21001, elbo bound per timestep: -1.072990
INFO:tensorflow:global_step/sec: 3.70431
INFO:tensorflow:Step 21101, elbo bound per timestep: -0.921844
INFO:tensorflow:Saving checkpoints for 21108 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69128
INFO:tensorflow:Step 21201, elbo bound per timestep: -1.064696
INFO:tensorflow:global_step/sec: 3.7886
INFO:tensorflow:Step 21301, elbo bound per timestep: -0.804586
INFO:tensorflow:global_step/sec: 3.74709
INFO:tensorflow:Step 21401, elbo bound per timestep: -0.734336
INFO:tensorflow:global_step/sec: 3.71272
INFO:tensorflow:Step 21501, elbo bound per timestep: -0.736721
INFO:tensorflow:Saving checkpoints for 21557 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.70331
INFO:tensorflow:Step 21601, elbo bound per timestep: -0.678197
INFO:tensorflow:global_step/sec: 3.65558
INFO:tensorflow:Step 21701, elbo bound per timestep: -1.100500
INFO:tensorflow:global_step/sec: 3.77741
INFO:tensorflow:Step 21801, elbo bound per timestep: -0.991786
INFO:tensorflow:global_step/sec: 3.72604
INFO:tensorflow:Step 21901, elbo bound per timestep: -0.890680
INFO:tensorflow:global_step/sec: 3.6717
INFO:tensorflow:Step 22001, elbo bound per timestep: -0.722860
INFO:tensorflow:Saving checkpoints for 22002 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.74037
INFO:tensorflow:Step 22101, elbo bound per timestep: -0.716064
INFO:tensorflow:global_step/sec: 3.74924
INFO:tensorflow:Step 22201, elbo bound per timestep: -0.687963
INFO:tensorflow:global_step/sec: 3.75015
INFO:tensorflow:Step 22301, elbo bound per timestep: -0.795578
INFO:tensorflow:global_step/sec: 3.73644
INFO:tensorflow:Step 22401, elbo bound per timestep: -0.694774
INFO:tensorflow:Saving checkpoints for 22452 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72703
INFO:tensorflow:Step 22501, elbo bound per timestep: -0.779019
INFO:tensorflow:global_step/sec: 3.71171
INFO:tensorflow:Step 22601, elbo bound per timestep: -0.744731
INFO:tensorflow:global_step/sec: 3.78004
INFO:tensorflow:Step 22701, elbo bound per timestep: -0.739943
INFO:tensorflow:global_step/sec: 3.74024
INFO:tensorflow:Step 22801, elbo bound per timestep: -0.761259
INFO:tensorflow:global_step/sec: 3.75396
INFO:tensorflow:Step 22901, elbo bound per timestep: -0.717537
INFO:tensorflow:Saving checkpoints for 22902 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68931
INFO:tensorflow:Step 23001, elbo bound per timestep: -0.715038
INFO:tensorflow:global_step/sec: 3.70411
INFO:tensorflow:Step 23101, elbo bound per timestep: -0.735971
INFO:tensorflow:global_step/sec: 3.74939
INFO:tensorflow:Step 23201, elbo bound per timestep: -0.703698
INFO:tensorflow:global_step/sec: 3.72392
INFO:tensorflow:Step 23301, elbo bound per timestep: -0.783196
INFO:tensorflow:Saving checkpoints for 23349 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71073
INFO:tensorflow:Step 23401, elbo bound per timestep: -0.781749
INFO:tensorflow:global_step/sec: 3.68194
INFO:tensorflow:Step 23501, elbo bound per timestep: -0.674284
INFO:tensorflow:global_step/sec: 3.69982
INFO:tensorflow:Step 23601, elbo bound per timestep: -0.774431
INFO:tensorflow:global_step/sec: 3.68719
INFO:tensorflow:Step 23701, elbo bound per timestep: -0.719700
INFO:tensorflow:Saving checkpoints for 23794 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75614
INFO:tensorflow:Step 23801, elbo bound per timestep: -0.738809
INFO:tensorflow:global_step/sec: 3.73096
INFO:tensorflow:Step 23901, elbo bound per timestep: -1.794747
INFO:tensorflow:global_step/sec: 3.81712
INFO:tensorflow:Step 24001, elbo bound per timestep: -1.002686
INFO:tensorflow:global_step/sec: 3.75991
INFO:tensorflow:Step 24101, elbo bound per timestep: -0.649911
INFO:tensorflow:global_step/sec: 3.73809
INFO:tensorflow:Step 24201, elbo bound per timestep: -0.805906
INFO:tensorflow:Saving checkpoints for 24245 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75688
INFO:tensorflow:Step 24301, elbo bound per timestep: -0.862584
INFO:tensorflow:global_step/sec: 3.72933
INFO:tensorflow:Step 24401, elbo bound per timestep: -0.667932
INFO:tensorflow:global_step/sec: 3.71885
INFO:tensorflow:Step 24501, elbo bound per timestep: -0.668128
INFO:tensorflow:global_step/sec: 3.75901
INFO:tensorflow:Step 24601, elbo bound per timestep: -0.732356
INFO:tensorflow:Saving checkpoints for 24694 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72636
INFO:tensorflow:Step 24701, elbo bound per timestep: -0.628415
INFO:tensorflow:global_step/sec: 3.77704
INFO:tensorflow:Step 24801, elbo bound per timestep: -0.695619
INFO:tensorflow:global_step/sec: 3.72559
INFO:tensorflow:Step 24901, elbo bound per timestep: -0.730392
INFO:tensorflow:global_step/sec: 3.82827
INFO:tensorflow:Step 25001, elbo bound per timestep: -0.619827
INFO:tensorflow:global_step/sec: 3.76327
INFO:tensorflow:Step 25101, elbo bound per timestep: -0.766918
INFO:tensorflow:Saving checkpoints for 25148 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75342
INFO:tensorflow:Step 25201, elbo bound per timestep: -0.601421
INFO:tensorflow:global_step/sec: 3.73277
INFO:tensorflow:Step 25301, elbo bound per timestep: -0.831466
INFO:tensorflow:global_step/sec: 3.70937
INFO:tensorflow:Step 25401, elbo bound per timestep: -0.648746
INFO:tensorflow:global_step/sec: 3.75318
INFO:tensorflow:Step 25501, elbo bound per timestep: -0.749604
INFO:tensorflow:Saving checkpoints for 25596 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72049
INFO:tensorflow:Step 25601, elbo bound per timestep: -0.721394
INFO:tensorflow:global_step/sec: 3.77817
INFO:tensorflow:Step 25701, elbo bound per timestep: -0.696055
INFO:tensorflow:global_step/sec: 3.69205
INFO:tensorflow:Step 25801, elbo bound per timestep: -0.594643
INFO:tensorflow:global_step/sec: 3.71318
INFO:tensorflow:Step 25901, elbo bound per timestep: -0.702147
INFO:tensorflow:global_step/sec: 3.7349
INFO:tensorflow:Step 26001, elbo bound per timestep: -0.756026
INFO:tensorflow:Saving checkpoints for 26043 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68466
INFO:tensorflow:Step 26101, elbo bound per timestep: -0.938721
INFO:tensorflow:global_step/sec: 3.79145
INFO:tensorflow:Step 26201, elbo bound per timestep: -1.137213
INFO:tensorflow:global_step/sec: 3.77405
INFO:tensorflow:Step 26301, elbo bound per timestep: -0.662706
INFO:tensorflow:global_step/sec: 3.76325
INFO:tensorflow:Step 26401, elbo bound per timestep: -0.632924
INFO:tensorflow:Saving checkpoints for 26494 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.70861
INFO:tensorflow:Step 26501, elbo bound per timestep: -0.729049
INFO:tensorflow:global_step/sec: 3.72041
INFO:tensorflow:Step 26601, elbo bound per timestep: -0.821361
INFO:tensorflow:global_step/sec: 3.78284
INFO:tensorflow:Step 26701, elbo bound per timestep: -0.596227
INFO:tensorflow:global_step/sec: 3.73078
INFO:tensorflow:Step 26801, elbo bound per timestep: -0.503825
INFO:tensorflow:global_step/sec: 3.80291
INFO:tensorflow:Step 26901, elbo bound per timestep: -0.606100
INFO:tensorflow:Saving checkpoints for 26945 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75667
INFO:tensorflow:Step 27001, elbo bound per timestep: -0.659573
INFO:tensorflow:global_step/sec: 3.7537
INFO:tensorflow:Step 27101, elbo bound per timestep: -0.757592
INFO:tensorflow:global_step/sec: 3.77395
INFO:tensorflow:Step 27201, elbo bound per timestep: -0.514000
INFO:tensorflow:global_step/sec: 3.73015
INFO:tensorflow:Step 27301, elbo bound per timestep: -0.575374
INFO:tensorflow:Saving checkpoints for 27395 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71642
INFO:tensorflow:Step 27401, elbo bound per timestep: -0.641147
INFO:tensorflow:global_step/sec: 3.74566
INFO:tensorflow:Step 27501, elbo bound per timestep: -0.597240
INFO:tensorflow:global_step/sec: 3.71925
INFO:tensorflow:Step 27601, elbo bound per timestep: -0.761037
INFO:tensorflow:global_step/sec: 3.73376
INFO:tensorflow:Step 27701, elbo bound per timestep: -0.645401
INFO:tensorflow:global_step/sec: 3.74407
INFO:tensorflow:Step 27801, elbo bound per timestep: -0.846569
INFO:tensorflow:Saving checkpoints for 27844 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.70452
INFO:tensorflow:Step 27901, elbo bound per timestep: -0.766050
INFO:tensorflow:global_step/sec: 3.71492
INFO:tensorflow:Step 28001, elbo bound per timestep: -0.623765
INFO:tensorflow:global_step/sec: 3.71128
INFO:tensorflow:Step 28101, elbo bound per timestep: -0.632352
INFO:tensorflow:global_step/sec: 3.73396
INFO:tensorflow:Step 28201, elbo bound per timestep: -0.692110
INFO:tensorflow:Saving checkpoints for 28291 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.74748
INFO:tensorflow:Step 28301, elbo bound per timestep: -0.748920
INFO:tensorflow:global_step/sec: 3.73464
INFO:tensorflow:Step 28401, elbo bound per timestep: -1.813460
INFO:tensorflow:global_step/sec: 3.73232
INFO:tensorflow:Step 28501, elbo bound per timestep: -0.834125
INFO:tensorflow:global_step/sec: 3.77559
INFO:tensorflow:Step 28601, elbo bound per timestep: -0.684359
INFO:tensorflow:global_step/sec: 3.75741
INFO:tensorflow:Step 28701, elbo bound per timestep: -0.913387
INFO:tensorflow:Saving checkpoints for 28740 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68307
INFO:tensorflow:Step 28801, elbo bound per timestep: -0.570199
INFO:tensorflow:global_step/sec: 3.77364
INFO:tensorflow:Step 28901, elbo bound per timestep: -0.775057
INFO:tensorflow:global_step/sec: 3.69728
INFO:tensorflow:Step 29001, elbo bound per timestep: -0.752278
INFO:tensorflow:global_step/sec: 3.72332
INFO:tensorflow:Step 29101, elbo bound per timestep: -0.820948
INFO:tensorflow:Saving checkpoints for 29189 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.74386
INFO:tensorflow:Step 29201, elbo bound per timestep: -0.689588
INFO:tensorflow:global_step/sec: 3.74494
INFO:tensorflow:Step 29301, elbo bound per timestep: -0.577212
INFO:tensorflow:global_step/sec: 3.73355
INFO:tensorflow:Step 29401, elbo bound per timestep: -0.877129
INFO:tensorflow:global_step/sec: 3.74441
INFO:tensorflow:Step 29501, elbo bound per timestep: -0.555399
INFO:tensorflow:global_step/sec: 3.80043
INFO:tensorflow:Step 29601, elbo bound per timestep: -0.650611
INFO:tensorflow:Saving checkpoints for 29640 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.78576
INFO:tensorflow:Step 29701, elbo bound per timestep: -0.655521
INFO:tensorflow:global_step/sec: 3.7805
INFO:tensorflow:Step 29801, elbo bound per timestep: -0.578889
INFO:tensorflow:global_step/sec: 3.7299
INFO:tensorflow:Step 29901, elbo bound per timestep: -0.848843
INFO:tensorflow:global_step/sec: 3.78002
INFO:tensorflow:Step 30001, elbo bound per timestep: -0.711337
INFO:tensorflow:Saving checkpoints for 30093 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.76909
INFO:tensorflow:Step 30101, elbo bound per timestep: -0.580901
INFO:tensorflow:global_step/sec: 3.70884
INFO:tensorflow:Step 30201, elbo bound per timestep: -0.624836
INFO:tensorflow:global_step/sec: 3.77919
INFO:tensorflow:Step 30301, elbo bound per timestep: -0.639884
INFO:tensorflow:global_step/sec: 3.76243
INFO:tensorflow:Step 30401, elbo bound per timestep: -0.712796
INFO:tensorflow:global_step/sec: 3.7315
INFO:tensorflow:Step 30501, elbo bound per timestep: -0.619797
INFO:tensorflow:Saving checkpoints for 30542 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.66982
INFO:tensorflow:Step 30601, elbo bound per timestep: -0.570065
INFO:tensorflow:global_step/sec: 3.78618
INFO:tensorflow:Step 30701, elbo bound per timestep: -0.714426
INFO:tensorflow:global_step/sec: 3.71904
INFO:tensorflow:Step 30801, elbo bound per timestep: -0.594839
INFO:tensorflow:global_step/sec: 3.80795
INFO:tensorflow:Step 30901, elbo bound per timestep: -0.569113
INFO:tensorflow:Saving checkpoints for 30994 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.76331
INFO:tensorflow:Step 31001, elbo bound per timestep: -0.780082
INFO:tensorflow:global_step/sec: 3.77905
INFO:tensorflow:Step 31101, elbo bound per timestep: -0.724367
INFO:tensorflow:global_step/sec: 3.7088
INFO:tensorflow:Step 31201, elbo bound per timestep: -0.461336
INFO:tensorflow:global_step/sec: 3.78757
INFO:tensorflow:Step 31301, elbo bound per timestep: -0.602580
INFO:tensorflow:global_step/sec: 3.75584
INFO:tensorflow:Step 31401, elbo bound per timestep: -1.010099
INFO:tensorflow:Saving checkpoints for 31444 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.73911
INFO:tensorflow:Step 31501, elbo bound per timestep: -0.617056
INFO:tensorflow:global_step/sec: 3.757
INFO:tensorflow:Step 31601, elbo bound per timestep: -0.721194
INFO:tensorflow:global_step/sec: 3.78255
INFO:tensorflow:Step 31701, elbo bound per timestep: -0.796281
INFO:tensorflow:global_step/sec: 3.69462
INFO:tensorflow:Step 31801, elbo bound per timestep: -0.653433
INFO:tensorflow:Saving checkpoints for 31895 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75376
INFO:tensorflow:Step 31901, elbo bound per timestep: -0.592026
INFO:tensorflow:global_step/sec: 3.78441
INFO:tensorflow:Step 32001, elbo bound per timestep: -0.613986
INFO:tensorflow:global_step/sec: 3.68261
INFO:tensorflow:Step 32101, elbo bound per timestep: -0.695021
INFO:tensorflow:global_step/sec: 3.80287
INFO:tensorflow:Step 32201, elbo bound per timestep: -0.895810
INFO:tensorflow:global_step/sec: 3.78097
INFO:tensorflow:Step 32301, elbo bound per timestep: -0.572393
INFO:tensorflow:Saving checkpoints for 32345 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.70225
INFO:tensorflow:Step 32401, elbo bound per timestep: -0.552712
INFO:tensorflow:global_step/sec: 3.73231
INFO:tensorflow:Step 32501, elbo bound per timestep: -0.738775
INFO:tensorflow:global_step/sec: 3.74808
INFO:tensorflow:Step 32601, elbo bound per timestep: -0.680045
INFO:tensorflow:global_step/sec: 3.7826
INFO:tensorflow:Step 32701, elbo bound per timestep: -0.606512
INFO:tensorflow:Saving checkpoints for 32794 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.66648
INFO:tensorflow:Step 32801, elbo bound per timestep: -0.604873
INFO:tensorflow:global_step/sec: 3.8072
INFO:tensorflow:Step 32901, elbo bound per timestep: -0.591178
INFO:tensorflow:global_step/sec: 3.73652
INFO:tensorflow:Step 33001, elbo bound per timestep: -0.650141
INFO:tensorflow:global_step/sec: 3.77411
INFO:tensorflow:Step 33101, elbo bound per timestep: -0.671777
INFO:tensorflow:global_step/sec: 3.7055
INFO:tensorflow:Step 33201, elbo bound per timestep: -0.658120
INFO:tensorflow:Saving checkpoints for 33244 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69189
INFO:tensorflow:Step 33301, elbo bound per timestep: -0.825685
INFO:tensorflow:global_step/sec: 3.77794
INFO:tensorflow:Step 33401, elbo bound per timestep: -0.712695
INFO:tensorflow:global_step/sec: 3.79593
INFO:tensorflow:Step 33501, elbo bound per timestep: -0.888601
INFO:tensorflow:global_step/sec: 3.77295
INFO:tensorflow:Step 33601, elbo bound per timestep: -0.635599
INFO:tensorflow:Saving checkpoints for 33697 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.76835
INFO:tensorflow:Step 33701, elbo bound per timestep: -0.666628
INFO:tensorflow:global_step/sec: 3.85535
INFO:tensorflow:Step 33801, elbo bound per timestep: -0.799061
INFO:tensorflow:global_step/sec: 3.71516
INFO:tensorflow:Step 33901, elbo bound per timestep: -0.768760
INFO:tensorflow:global_step/sec: 3.71718
INFO:tensorflow:Step 34001, elbo bound per timestep: -0.662166
INFO:tensorflow:global_step/sec: 3.73007
INFO:tensorflow:Step 34101, elbo bound per timestep: -0.865856
INFO:tensorflow:Saving checkpoints for 34147 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68882
INFO:tensorflow:Step 34201, elbo bound per timestep: -0.538440
INFO:tensorflow:global_step/sec: 3.76556
INFO:tensorflow:Step 34301, elbo bound per timestep: -0.655112
INFO:tensorflow:global_step/sec: 3.77276
INFO:tensorflow:Step 34401, elbo bound per timestep: -0.540338
INFO:tensorflow:global_step/sec: 3.71549
INFO:tensorflow:Step 34501, elbo bound per timestep: -0.567046
INFO:tensorflow:Saving checkpoints for 34597 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.78318
INFO:tensorflow:Step 34601, elbo bound per timestep: -0.616814
INFO:tensorflow:global_step/sec: 3.72062
INFO:tensorflow:Step 34701, elbo bound per timestep: -0.524634
INFO:tensorflow:global_step/sec: 3.75047
INFO:tensorflow:Step 34801, elbo bound per timestep: -0.673076
INFO:tensorflow:global_step/sec: 3.71302
INFO:tensorflow:Step 34901, elbo bound per timestep: -0.572902
INFO:tensorflow:global_step/sec: 3.81042
INFO:tensorflow:Step 35001, elbo bound per timestep: -0.571084
INFO:tensorflow:Saving checkpoints for 35047 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69113
INFO:tensorflow:Step 35101, elbo bound per timestep: -0.676653
INFO:tensorflow:global_step/sec: 3.75816
INFO:tensorflow:Step 35201, elbo bound per timestep: -0.693583
INFO:tensorflow:global_step/sec: 3.7477
INFO:tensorflow:Step 35301, elbo bound per timestep: -0.634695
INFO:tensorflow:global_step/sec: 3.71635
INFO:tensorflow:Step 35401, elbo bound per timestep: -0.670400
INFO:tensorflow:Saving checkpoints for 35496 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.77961
INFO:tensorflow:Step 35501, elbo bound per timestep: -0.616414
INFO:tensorflow:global_step/sec: 3.766
INFO:tensorflow:Step 35601, elbo bound per timestep: -0.721086
INFO:tensorflow:global_step/sec: 3.79167
INFO:tensorflow:Step 35701, elbo bound per timestep: -0.771639
INFO:tensorflow:global_step/sec: 3.71852
INFO:tensorflow:Step 35801, elbo bound per timestep: -0.610783
INFO:tensorflow:global_step/sec: 3.78557
INFO:tensorflow:Step 35901, elbo bound per timestep: -0.595350
INFO:tensorflow:Saving checkpoints for 35950 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.75929
INFO:tensorflow:Step 36001, elbo bound per timestep: -0.803556
INFO:tensorflow:global_step/sec: 3.77417
INFO:tensorflow:Step 36101, elbo bound per timestep: -0.580081
INFO:tensorflow:global_step/sec: 3.74921
INFO:tensorflow:Step 36201, elbo bound per timestep: -0.632336
INFO:tensorflow:global_step/sec: 3.67566
INFO:tensorflow:Step 36301, elbo bound per timestep: -0.563664
INFO:tensorflow:Saving checkpoints for 36397 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72326
INFO:tensorflow:Step 36401, elbo bound per timestep: -0.770496
INFO:tensorflow:global_step/sec: 3.76306
INFO:tensorflow:Step 36501, elbo bound per timestep: -0.606784
INFO:tensorflow:global_step/sec: 3.69538
INFO:tensorflow:Step 36601, elbo bound per timestep: -0.583560
INFO:tensorflow:global_step/sec: 3.79037
INFO:tensorflow:Step 36701, elbo bound per timestep: -0.748704
INFO:tensorflow:global_step/sec: 3.77505
INFO:tensorflow:Step 36801, elbo bound per timestep: -0.493782
INFO:tensorflow:Saving checkpoints for 36847 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68947
INFO:tensorflow:Step 36901, elbo bound per timestep: -0.797269
INFO:tensorflow:global_step/sec: 3.71344
INFO:tensorflow:Step 37001, elbo bound per timestep: -0.562445
INFO:tensorflow:global_step/sec: 3.70708
INFO:tensorflow:Step 37101, elbo bound per timestep: -0.724245
INFO:tensorflow:global_step/sec: 3.79891
INFO:tensorflow:Step 37201, elbo bound per timestep: -0.569017
INFO:tensorflow:Saving checkpoints for 37296 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.72116
INFO:tensorflow:Step 37301, elbo bound per timestep: -0.563803
INFO:tensorflow:global_step/sec: 3.71733
INFO:tensorflow:Step 37401, elbo bound per timestep: -0.587242
INFO:tensorflow:global_step/sec: 3.7446
INFO:tensorflow:Step 37501, elbo bound per timestep: -0.552587
INFO:tensorflow:global_step/sec: 3.74046
INFO:tensorflow:Step 37601, elbo bound per timestep: -0.852171
INFO:tensorflow:global_step/sec: 3.70583
INFO:tensorflow:Step 37701, elbo bound per timestep: -0.958163
INFO:tensorflow:Saving checkpoints for 37743 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.64372
INFO:tensorflow:Step 37801, elbo bound per timestep: -0.631223
INFO:tensorflow:global_step/sec: 3.69294
INFO:tensorflow:Step 37901, elbo bound per timestep: -0.651640
INFO:tensorflow:global_step/sec: 3.73729
INFO:tensorflow:Step 38001, elbo bound per timestep: -0.523597
INFO:tensorflow:global_step/sec: 3.74268
INFO:tensorflow:Step 38101, elbo bound per timestep: -0.738810
INFO:tensorflow:Saving checkpoints for 38190 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.7796
INFO:tensorflow:Step 38201, elbo bound per timestep: -0.690976
INFO:tensorflow:global_step/sec: 3.73535
INFO:tensorflow:Step 38301, elbo bound per timestep: -0.760125
INFO:tensorflow:global_step/sec: 3.7821
INFO:tensorflow:Step 38401, elbo bound per timestep: -0.648441
INFO:tensorflow:global_step/sec: 3.75846
INFO:tensorflow:Step 38501, elbo bound per timestep: -0.631667
INFO:tensorflow:global_step/sec: 3.76934
INFO:tensorflow:Step 38601, elbo bound per timestep: -0.543154
INFO:tensorflow:Saving checkpoints for 38641 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.68601
INFO:tensorflow:Step 38701, elbo bound per timestep: -0.535370
INFO:tensorflow:global_step/sec: 3.69835
INFO:tensorflow:Step 38801, elbo bound per timestep: -0.562232
INFO:tensorflow:global_step/sec: 3.81783
INFO:tensorflow:Step 38901, elbo bound per timestep: -0.651454
INFO:tensorflow:global_step/sec: 3.77194
INFO:tensorflow:Step 39001, elbo bound per timestep: -0.495198
INFO:tensorflow:Saving checkpoints for 39089 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.69094
INFO:tensorflow:Step 39101, elbo bound per timestep: -0.735205
INFO:tensorflow:global_step/sec: 3.79523
INFO:tensorflow:Step 39201, elbo bound per timestep: -0.717831
INFO:tensorflow:global_step/sec: 3.78566
INFO:tensorflow:Step 39301, elbo bound per timestep: -0.755740
INFO:tensorflow:global_step/sec: 3.72071
INFO:tensorflow:Step 39401, elbo bound per timestep: -0.663696
INFO:tensorflow:global_step/sec: 3.66311
INFO:tensorflow:Step 39501, elbo bound per timestep: -0.728133
INFO:tensorflow:Saving checkpoints for 39540 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.77175
INFO:tensorflow:Step 39601, elbo bound per timestep: -0.538489
INFO:tensorflow:global_step/sec: 3.7333
INFO:tensorflow:Step 39701, elbo bound per timestep: -0.776719
INFO:tensorflow:global_step/sec: 3.69775
INFO:tensorflow:Step 39801, elbo bound per timestep: -0.621886
INFO:tensorflow:global_step/sec: 3.77143
INFO:tensorflow:Step 39901, elbo bound per timestep: -0.740274
INFO:tensorflow:Saving checkpoints for 39988 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
INFO:tensorflow:global_step/sec: 3.71852
INFO:tensorflow:Step 40001, elbo bound per timestep: -0.671975
INFO:tensorflow:Saving checkpoints for 40001 into ./chkpt/elbo-aisdk_20180208_train.pkl-data_dim-402-latent_size-100-batch_size-32/model.ckpt.
Global step:  40001
```

#### Step 2: Running task-specific submodels

In [2]:
import glob
import pickle

with open(config.testset_path, "rb") as f:
    Vs_test = pickle.load(f)
dataset_size = len(Vs_test)


In [ ]:
step = None

tf.Graph().as_default()
global_step = tf.train.get_or_create_global_step()
inputs, targets, mmsis, time_starts, time_ends, lengths, model = (
    runners.create_dataset_and_model(config, shuffle=False, repeat=False)
)

if config.mode == "traj_reconstruction":
    config.missing_data = True
# else:
#    config.missing_data = False

track_sample, track_true, log_weights, ll_per_t, ll_acc, _, _, _ = (
    runners.create_eval_graph(inputs, targets, lengths, model, config)
)
saver = tf.train.Saver()
sess = tf.train.SingularMonitoredSession()
runners.wait_for_checkpoint(saver, sess, config.logdir)
step = sess.run(global_step)

# runners.wait_for_checkpoint(saver, sess, config.logdir)
# step = sess.run(global_step)
# print(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))

if step is None:
    # The log filename contains the step.
    index_filename = sorted(glob.glob(config.logdir+"/*.index"))[-1] # the lastest step
    step = int(index_filename.split(".index")[0].split("ckpt-")[-1])
    

print("Global step: ", step)
outputs_path = "results/"\
            + config.trainingset_path.split("/")[-2] + "/"\
            + "logprob-"\
            + os.path.basename(config.trainingset_name) + "-"\
            + os.path.basename(config.testset_name) + "-"\
            + str(config.latent_size)\
            + "-missing_data-" + str(config.missing_data)\
            + "-step-"+str(step)\
            + ".pkl"
if not os.path.exists(os.path.dirname(outputs_path)):
    os.makedirs(os.path.dirname(outputs_path))

save_dir = "results/"\
            + config.trainingset_path.split("/")[-2] + "/"\
            + "local_logprob-"\
            + os.path.basename(config.trainingset_name) + "-"\
            + os.path.basename(config.testset_name).replace("test","valid") + "-"\
            + str(config.latent_size) + "-"\
            + "missing_data-" + str(config.missing_data)\
            + "-step-"+str(step)\
            +"/"     

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
FIG_DPI = 150

l_dict = []
for d_i in tqdm(list(range(math.ceil(dataset_size / config.batch_size)))):
    inp, tar, mmsi, t_start, t_end, seq_len, log_weights_np, true_np, ll_t = (
        sess.run(
            [
                inputs,
                targets,
                mmsis,
                time_starts,
                time_ends,
                lengths,
                log_weights,
                track_true,
                ll_per_t,
            ]
        )
    )
    for d_idx_inbatch in range(inp.shape[1]):
        D = dict()
        seq_len_d = seq_len[d_idx_inbatch]
        D["seq"] = np.nonzero(tar[:seq_len_d, d_idx_inbatch, :])[1].reshape(-1, 4)
        D["t_start"] = t_start[d_idx_inbatch]
        D["t_end"] = t_end[d_idx_inbatch]
        D["mmsi"] = mmsi[d_idx_inbatch]
        D["log_weights"] = log_weights_np[:seq_len_d, :, d_idx_inbatch]
        l_dict.append(D)
with open(outputs_path, "wb") as f:
    pickle.dump(l_dict, f)

""" LL
Plot the distribution of log[p(x_t|h_t)] of each track in the test set.
"""

v_logprob = np.empty((0,))
v_logprob_stable = np.empty((0,))

count = 0
for D in tqdm(l_dict):
    log_weights_np = D["log_weights"]
    ll_t = np.mean(log_weights_np)
    v_logprob = np.concatenate((v_logprob, [ll_t]))

d_mean = np.mean(v_logprob)
d_std = np.std(v_logprob)
d_thresh = d_mean - 3 * d_std

plt.figure(figsize=(1920/FIG_DPI, 640/FIG_DPI), dpi=FIG_DPI)
plt.plot(v_logprob,'o')
plt.title("Log likelihood " + os.path.basename(config.testset_name)\
            + ", mean = {0:02f}, std = {1:02f}, threshold = {2:02f}".format(d_mean, d_std, d_thresh))
plt.plot([0,len(v_logprob)], [d_thresh, d_thresh],'r')

plt.xlim([0,len(v_logprob)])
fig_name = "results/"\
        + config.trainingset_path.split("/")[-2] + "/" \
        + "logprob-" \
        + config.bound + "-"\
        + os.path.basename(config.trainingset_name) + "-"\
        + os.path.basename(config.testset_name)\
        + "-latent_size-" + str(config.latent_size)\
        + "-ll_thresh" + str(round(d_thresh, 2))\
        + "-missing_data-" + str(config.missing_data)\
        + "-step-"+str(step)\
        + ".png"
plt.savefig(fig_name,dpi = FIG_DPI)
plt.close()

![](fig_name)

In [ ]:
from IPython.display import Image
Image(filename=fig_name) 